In [24]:
import requests
import csv
from datetime import datetime, timedelta
import time

API_LIMIT_PER_MINUTE = 90  # Bitfinex REST API limit
REQUEST_INTERVAL = 60 / API_LIMIT_PER_MINUTE  # Time between requests in seconds

In [26]:
def save_to_csv(data, filename="eth_prices_15s.csv"):
    """Save aggregated data to a CSV file."""
    with open(filename, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Timestamp", "Price (USD)"])
        for row in data:
            timestamp = datetime.utcfromtimestamp(row[0] / 1000).strftime('%Y-%m-%d %H:%M:%S')
            writer.writerow([timestamp, row[1]])
    print(f"Data saved to {filename}")

In [27]:
def fetch_bitfinex_trades(start, end, symbol="tETHUSD"):
    """Fetch trades from Bitfinex within a given time range."""
    url = f"https://api-pub.bitfinex.com/v2/trades/{symbol}/hist"
    params = {
        "start": start,
        "end": end,
        "sort": 1,  # Ascending order
        "limit": 10000  # Max limit per request
    }
    response = requests.get(url, params=params)

    # Handle Too Many Requests (Rate Limit)
    if response.status_code == 429:  # Too Many Requests
        print(f"Rate limit hit! Waiting for 1 minute... \r\n Response: {response}")
        time.sleep(60)  # Wait for 1 minute
        return fetch_bitfinex_trades(start, end)  # Retry after waiting

    response.raise_for_status()
    return response.json()

In [29]:
end_date = datetime.utcnow()
start_date = end_date - timedelta(days=2)

start_timestamp = int(start_date.timestamp() * 1000)
end_timestamp = int(end_date.timestamp() * 1000)

all_trades = []
current_start = start_timestamp
request_count = 0

# Fetch data in chunks to handle API limits
while current_start < end_timestamp:
    print(f"Day {current_start}")
    next_end = min(current_start + 86400000, end_timestamp)  # Fetch 1 day of data at a time
    try:
        trades = fetch_bitfinex_trades(current_start, next_end)
        all_trades.extend(trades)
        request_count += 1

        # Throttle requests to avoid rate limits
        if request_count >= API_LIMIT_PER_MINUTE:
            print("API limit reached, sleeping for 1 minute...")
            time.sleep(60)  # Wait for 1 minute
            request_count = 0
        else:
            time.sleep(REQUEST_INTERVAL)

        current_start = next_end
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        print("Waiting for 1 minute before retrying...")
        time.sleep(60)  # Wait for 1 minute on error and continue

/var/folders/1f/5lhk7sb97rz53v7gk1vkcjkr0000gn/T/ipykernel_25784/454803546.py:1: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end_date = datetime.utcnow()


Day 1737188315645
Day 1737274715645


ValueError: too many values to unpack (expected 3)

In [30]:
def aggregate_to_15s_intervals(trades):
    """Aggregate trade data into 15-second intervals."""
    aggregated = []
    current_interval_start = None
    prices = []

    for trade in trades:
        print(trade)
        timestamp, price = trade
        # Convert to 15-second intervals
        interval_start = timestamp - (timestamp % 15000)

        if current_interval_start is None:
            current_interval_start = interval_start

        if interval_start != current_interval_start:
            # Calculate average price for the interval
            avg_price = sum(prices) / len(prices)
            aggregated.append([current_interval_start, avg_price])
            # Reset for next interval
            current_interval_start = interval_start
            prices = []

        prices.append(price)

    # Add the last interval
    if prices:
        avg_price = sum(prices) / len(prices)
        aggregated.append([current_interval_start, avg_price])

    return aggregated

In [31]:
# Aggregate data into 15-second intervals
aggregated_data = aggregate_to_15s_intervals(all_trades)

# Save to CSV
save_to_csv(aggregated_data)

ValueError: too many values to unpack (expected 2)